### Finding best Minimally Complex model for supreme court voting dataset using integer representation

In [1]:
# Imports
import sys
sys.path.insert(0, '../')
from src.utils import *
from src.mcm_binary import mcm

In [2]:
model = mcm('../data/SC_voting/US_SupremeCourt_n9_N895.txt')

In [3]:
# Evidence of IM with original basis (Should be -5258)
print('Independent model in the original basis')
print('Evidence: ', model.calc_log_evidence(model.mcms[-1]))

Independent model in the original basis
Evidence:  -5258.100240438084


In [4]:
# Finding best MCM in original basis (Should be [[0, 2, 3, 4, 6], [1, 5, 7, 8]] with evidence -3300.4)
model.find_best_mcm()
print('Best MCM in original basis (Exhaustive search)')
print('MCM: ', model.best_mcm)
print('Evidence: ', model.best_evidence)

Best MCM in original basis (Exhaustive search)
MCM:  [[0, 2, 3, 4, 6], [1, 5, 7, 8]]
Evidence:  -3300.395469673639


In [5]:
# Finding best MCM in original basis using a Greedy search (Should be [[0, 2, 3, 4, 6], [1, 5, 7, 8]] with evidence -3300.4)
model.find_best_mcm(method='greedy')
print('Best MCM in original basis (Greedy search)')
print('MCM: ', model.best_mcm)
print('Evidence: ', model.best_evidence)

Best MCM in original basis (Greedy search)
MCM:  [[0, 4, 2, 3, 6], [1, 5, 7, 8]]
Evidence:  -3300.395469673639


In [6]:
# Finding best MCM in original basis using the divide and conquer scheme
model.find_best_mcm(method='divide_and_conquer')
print('Best MCM in original basis (Divide and conquer)')
print('MCM: ', model.best_mcm)
print('Evidence: ', model.best_evidence)

Best MCM in original basis (Divide and conquer)
MCM:  [[0, 2, 3, 4, 6], [1, 8, 7, 5]]
Evidence:  -3300.395469673639


In [7]:
# Finding the best IM (Should be [72, 160, 384, 17, 65, 5, 130, 260, 64] with evidence -3327)
model.find_best_im(max_interactions=2)
print('Best Independent model')
print('IM: ', model.best_im)
print('Evidence: ', model.calc_log_evidence(model.mcms[-1]))

Best Independent model
IM:  [ 72 160 384  17  65   5 130 260  64]
Evidence:  -3327.079945846724


In [8]:
# Finding best MCM in optimal basis (Should be [[0], [1, 2, 6], [3, 4, 5, 7, 8]] with evidence -3154.4)
model.find_best_mcm()
print('Best MCM in the optimal basis (Exhaustive search)')
print('MCM: ', model.best_mcm)
print('Evidence: ', model.best_evidence)

Best MCM in the optimal basis (Exhaustive search)
MCM:  [[0], [1, 2, 6], [3, 4, 5, 7, 8]]
Evidence:  -3154.421230299754


In [9]:
# Finding the best MCM in the optimal basis using Greedy search ([[0, 4, 3, 5], [1, 2, 6, 7, 8]] -3163.252527450342)
model.find_best_mcm(method='greedy')
print('Best MCM in the optimal basis (Greedy search)')
print('MCM: ', model.best_mcm)
print('Evidence: ', model.best_evidence)

Best MCM in the optimal basis (Greedy search)
MCM:  [[0, 4, 3, 5], [1, 2, 6, 7, 8]]
Evidence:  -3163.252527450342


In [10]:
# Finding the best MCM in the optimal basis using the divide and conquer scheme
model.find_best_mcm(method='divide_and_conquer')
print('Best MCM in the optimal basis (Divide and conquer)')
print('MCM: ', model.best_mcm)
print('Evidence: ', model.best_evidence)

Best MCM in the optimal basis (Divide and conquer)
MCM:  [[1, 2, 6], [3, 5, 4, 7, 8], [0]]
Evidence:  -3154.421230299755


### Generating samples from spin model with discrete states

In [20]:
q = 3
n = 2

In [21]:
# Construct random array for parameters (range?)
g = np.random.uniform(low=-1, high=1, size=q**n)

In [22]:
def construct_s(q):
    S = np.ones((q, q), dtype=complex)
    for i in range(q):
        for j in range(i, q):
            S[i,j] = np.exp(-((i*j)*2j*np.pi) / q)
            S[j,i] = S[i,j]
    return S

In [23]:
S = construct_s(q)
S_matrix = np.copy(S)
for _ in range(n-1):
    S_matrix = np.kron(S_matrix, S)

In [24]:
S

array([[ 1. +0.j       ,  1. +0.j       ,  1. +0.j       ],
       [ 1. +0.j       , -0.5-0.8660254j, -0.5+0.8660254j],
       [ 1. +0.j       , -0.5+0.8660254j, -0.5-0.8660254j]])

In [25]:
S_matrix

array([[ 1. +0.00000000e+00j,  1. +0.00000000e+00j,  1. +0.00000000e+00j,
         1. +0.00000000e+00j,  1. +0.00000000e+00j,  1. +0.00000000e+00j,
         1. +0.00000000e+00j,  1. +0.00000000e+00j,  1. +0.00000000e+00j],
       [ 1. +0.00000000e+00j, -0.5-8.66025404e-01j, -0.5+8.66025404e-01j,
         1. +0.00000000e+00j, -0.5-8.66025404e-01j, -0.5+8.66025404e-01j,
         1. +0.00000000e+00j, -0.5-8.66025404e-01j, -0.5+8.66025404e-01j],
       [ 1. +0.00000000e+00j, -0.5+8.66025404e-01j, -0.5-8.66025404e-01j,
         1. +0.00000000e+00j, -0.5+8.66025404e-01j, -0.5-8.66025404e-01j,
         1. +0.00000000e+00j, -0.5+8.66025404e-01j, -0.5-8.66025404e-01j],
       [ 1. +0.00000000e+00j,  1. +0.00000000e+00j,  1. +0.00000000e+00j,
        -0.5-8.66025404e-01j, -0.5-8.66025404e-01j, -0.5-8.66025404e-01j,
        -0.5+8.66025404e-01j, -0.5+8.66025404e-01j, -0.5+8.66025404e-01j],
       [ 1. +0.00000000e+00j, -0.5-8.66025404e-01j, -0.5+8.66025404e-01j,
        -0.5-8.66025404e-01j, -0.5

In [90]:
p = np.exp(S_matrix @ g)

In [92]:
# Imaginary probabilities
p /= np.sum(p)
p

array([ 0.02305325-6.16336856e-17j,  0.00243038-9.64589741e-03j,
        0.00243038+9.64589741e-03j, -0.00350986+1.62851899e-02j,
        0.01281934+3.84899618e-03j,  0.47673352+8.51890406e-02j,
       -0.00350986-1.62851899e-02j,  0.47673352-8.51890406e-02j,
        0.01281934-3.84899618e-03j])

In [63]:
tmp = [[1,2,3], [2,3,4], [3,4,5]]

In [64]:
current = tmp
new = copy.deepcopy(current)
new[0] = [44,55,66]

In [65]:
tmp = new

In [66]:
tmp

[[44, 55, 66], [2, 3, 4], [3, 4, 5]]

In [67]:
current

[[1, 2, 3], [2, 3, 4], [3, 4, 5]]